Apply attention.
MOdified from dual attention rnn from https://chandlerzuo.github.io/blog/2017/11/darnn

These are seq2seq model which predicts final status sequentially.  
This is not good because we get errors from each prediction step.  

Another thing to note is  
In case prediction_period is larger than 1,
we do not need to infer all information.  
We have information before predicting dates.  
So y market data before predicting date can be used in inference.  

Results are bad for distant prediction.

## Load data

In [653]:
import copy as cp
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt

In [4]:
import os, itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import sys
try: # ip: 153
    lib_s = '/home/adminuser/public/libs/pytorch_examples/basic_networks'
    sys.path.append(lib_s)
    import generate_model as nets
except: # ip : 103
    lib_s = '/home/bwlee/work/codes/examples/basic_networks'
    sys.path.append(lib_s)
    import generate_model as nets

## Make input data
interest.csv has only closed price.   
On the other hand, zipline needs OLHCV(?) format for each index  
make dummy columns and put 'close' value to them  
make csv file for each index  

In [5]:
f_s0 = 'interest.csv'
df0 = pd.read_csv(f_s0, index_col='date')
df0 = df0.iloc[::-1] # 과거에서 현재로 정렬순서 변경

In [643]:
# normalized input, norm val, tgt_date, tgt_val, tgt_diff_ratio, 
class Environ():
    def __init__(self):
        self.indices = ['trea3', 'trea5', 'trea10']
        #self.tgt_index = 'trea10'
        self.tgt_index = 'trea3'
        # input window size
        self.input_size = 15
        # prediction is made after algo.pred_period from the present
        self.pred_period = 5

env = Environ() # setting values

In [644]:
dff0 = df0
dff = dff0

In [645]:
diff_periods = [int(env.input_size/4*x) for x in [1, 2, 3]]
for diff_period in diff_periods:
    diff = dff0.diff(periods=diff_period)
    ii = diff_period
    columns = ['d%d_diff_trea3'%(ii), 'd%d_diff_trea5'%(ii), 'd%d_diff_trea10'%(ii)]
    diff.columns = columns
    dff = dff.merge(diff, how='inner', on='date')
dff['diff35'] = dff['trea3'] - dff['trea5']
dff['diff310'] = dff['trea3'] - dff['trea10']
dff['diff510'] = dff['trea5'] - dff['trea10']

dff = dff.dropna()

## prepare data without time confusion
Seperate two actions

In [646]:
def push_data(env, data, mkt_data):
    """
    read past data and push them to mkt_data
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    for ii_days in range(len(data)):
        if ii_days+1 < env.input_size:
            continue
        elif ii_days+1 == env.input_size:
            env.start_date = data.iloc[ii_days].name
        
        predicting_date = data.iloc[ii_days].name
        # historical data from today to env.input_size behind days
        history = data.iloc[ii_days+1-env.input_size:ii_days+1]
        
        ## normalize
        """
        #  : look back 1 year
        lookback_step = 250
        lookback_step = env.input_size
        if ii_days+1 < lookback_step:
            cumul = data.iloc[:ii_days+1]
        else:
            cumul = data.iloc[ii_days+1-lookback_step:ii_days+1]
        
        mean_val = cumul.mean() 
        range_val = 2*cumul.std()
        """
        
        mean_val = history.mean() 
        range_val = history.max() - history.min()

        norm_history = (history-mean_val)/range_val
        mkt_data[predicting_date] = [norm_history, mean_val, range_val, 
                                     None, None, None, None, None]
        
            
def push_tgt(env, data, mkt_data):
    """
    read future data and push them to mkt_data
    future_prediction_date is a future date when model want to predict
    past_predicting_date is a past date when model made a prediction with given information
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    for ii_days in range(len(data)):
        ii_predicting = ii_days - env.pred_period
        predicting_date = data.iloc[ii_predicting].name
        if ii_predicting < 0 or predicting_date < env.start_date:
            continue
        
        past_history = mkt_data[predicting_date][0]
        mean_val = mkt_data[predicting_date][1]
        range_val = mkt_data[predicting_date][2]
        
        past_price = (past_history.iloc[-1]*range_val+mean_val)[env.tgt_index] # 49.6
        # wrong
        #past_priceb = (past_history.iloc[-env.pred_period:]*range_val+mean_val)[env.tgt_index] 
        #print('past_priceb', past_priceb)
        #past_price = (past_history.iloc[-5]*range_val+mean_val)[env.tgt_index] # 56.3
        
        #mkt_data[predicting_date][0] = past_history[:-4]
        
        #past_price = (past_history.iloc[-3]*range_val+mean_val)[env.tgt_index] # 53.0, # 53.0
        #past_price = (past_history.iloc[-13]*range_val+mean_val)[env.tgt_index] # 67.4
        #past_price = (past_history*range_val+mean_val).mean()[env.tgt_index] # 58.4
        #past_price = (past_history*range_val+mean_val)[:-3].mean()[env.tgt_index] # 63.2
        #past_price = (past_history*range_val+mean_val)[:-5].mean()[env.tgt_index] # 63.7
        
        #past_price = (past_history.iloc[-10]*range_val+mean_val)[env.tgt_index] # 66.4

        #past_price = (past_history.iloc[-5:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        #past_price = (past_history.iloc[-3:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        
        #past_price = (past_history.iloc[-10:].mean()*range_val+mean_val)[env.tgt_index] # 55.9
        #past_price = (past_history.iloc[-10:-5].mean()*range_val+mean_val)[env.tgt_index] # 60.4
        
        prediction_date = data.iloc[ii_days].name
        tgt_price = data.iloc[ii_days][env.tgt_index]
        # tgt should have one more time step than input, because is has initial value
        
        #tgt_price = (data.iloc[ii_days-1:ii_days+2].mean())[env.tgt_index]
        
        # ignore dates and subtract to get target history
        tgt_ratio = (tgt_price - past_price)/past_price*10
        
        mkt_data[predicting_date][3] = prediction_date
        mkt_data[predicting_date][4] = np.single(tgt_price)
        mkt_data[predicting_date][5] = np.single(tgt_ratio)
        
        mkt_data[predicting_date][6] = np.single(past_price)
        mkt_data[predicting_date][7] = predicting_date

### Push past data

In [647]:
mkt_data = {} # object to store data for learning
push_data(env, dff, mkt_data)

### Check data are pushed correctly
* no dates should be ahead of key date
* prediction info are not pushed yet

In [30]:
print(list(mkt_data.keys())[:3])

['2000-11-11', '2000-11-13', '2000-11-14']


In [676]:
mkt_data['2000-11-23']

[               trea3     trea5    trea10  d3_diff_trea3  d3_diff_trea5  \
 date                                                                     
 2000-11-07  0.681934  0.687143  0.711795       0.130667       0.124551   
 2000-11-08  0.567430  0.558571  0.588718       0.030667       0.016766   
 2000-11-09  0.124682  0.080000  0.111795      -0.382667      -0.414371   
 2000-11-10 -0.226463 -0.212857 -0.149744      -0.689333      -0.665868   
 2000-11-11 -0.226463 -0.212857 -0.149744      -0.589333      -0.558084   
 2000-11-13 -0.318066 -0.305714 -0.288205      -0.282667      -0.234731   
 2000-11-14 -0.287532 -0.312857 -0.288205       0.050667       0.004790   
 2000-11-15 -0.150127 -0.191429 -0.172821       0.170667       0.106587   
 2000-11-16 -0.081425 -0.120000 -0.134359       0.310667       0.244311   
 2000-11-17 -0.104326 -0.141429 -0.157436       0.264000       0.232335   
 2000-11-18 -0.104326 -0.141429 -0.157436       0.144000       0.130539   
 2000-11-20 -0.073791 -0.

In [562]:
env.start_date

'2000-11-23'

### Push future data

In [677]:
push_tgt(env, dff, mkt_data)
mkt_data = {key: val for key, val in mkt_data.items() if (val[-1] is not None)}

### CHECK tgt push is correctly made
end value of '2000-11-23' * norm == prediction value 

In [226]:
print(mkt_data['2000-11-11'])
print(mkt_data['2000-11-23'])

[               trea3     trea5    trea10
date                                    
2000-10-26  0.300000  0.272099  0.293963
2000-10-27  0.105970  0.131358  0.152231
2000-10-28  0.105970  0.131358  0.152231
2000-10-30  0.091045  0.123951  0.152231
2000-10-31  0.150746  0.175802  0.183727
2000-11-01  0.105970  0.123951  0.136483
2000-11-02  0.128358  0.123951  0.104987
2000-11-03  0.158209  0.160988  0.136483
2000-11-04  0.158209  0.160988  0.136483
2000-11-06  0.188060  0.198025  0.175853
2000-11-07  0.188060  0.205432  0.175853
2000-11-08  0.076119  0.072099  0.049869
2000-11-09 -0.356716 -0.424198 -0.438320
2000-11-10 -0.700000 -0.727901 -0.706037
2000-11-11 -0.700000 -0.727901 -0.706037, trea3     7.664000
trea5     7.866333
trea10    8.143333
dtype: float64, trea3     0.670
trea5     0.675
trea10    0.635
dtype: float64, '2000-12-05', 7.07, -1.0108074]
[               trea3     trea5    trea10
date                                    
2000-11-07  0.681934  0.687143  0.711795
2000-11-

In [1]:
## Training
### Data preparation

In [682]:
class Index_data(Dataset):
    def __init__(self, in_data0, dates, addi=None):
        """
        :param in_data: dict of (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        :param addi: (optional) additional non-sequential elements that can be added to in_data
        """
        in_data = cp.deepcopy(in_data0)
        if addi is not None:
            assert list(in_data.keys()) == list(addi.keys())
            
        # change dataframe to float numpy array
        for key, val in in_data.items():
            # transform in_data into Tensor
            if not 'Tensor' in str(type(in_data[key][0])) :
                in_data[key][0] = torch.tensor(in_data[key][0].to_numpy(np.single))
                in_data[key][1] = torch.tensor(in_data[key][1].to_numpy(np.single))
                in_data[key][2] = torch.tensor(in_data[key][2].to_numpy(np.single))
                #in_data[key][4] = torch.tensor(in_data[key][4])
                #in_data[key][5] = torch.tensor(in_data[key][5]) # isn't this necessary? not in rnn and cnn model
            
            if addi is not None:
                in_data[key].append(addi[key])
            else:
                # None is not accept in Dataloader, so give null value
                in_data[key].append('NULL')
                
        self.in_data = in_data
        self.i2dates = {}
        for ii, date1 in enumerate(dates):
             self.i2dates[ii] = date1
        
    def __getitem__(self, ii_date):
        """
        :return: (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        """
        predicting_date = self.i2dates[ii_date]
        return self.in_data[predicting_date]
    
    def __len__(self):
        return len(self.in_data)

In [683]:
#erase data that do not have prediction target
traintest_dates = sorted(list(mkt_data.keys()))
n_train = len(traintest_dates)*8//10
n_test = len(traintest_dates) - n_train
train_dates = traintest_dates[:n_train]
test_dates = traintest_dates[n_train:]

train_data = {}
test_data = {}
for key, val in mkt_data.items():
    if key in train_dates:
        train_data[key] = val
    elif key in test_dates:
        test_data[key] = val

In [684]:
## this is an example, I already add differences
def get_addi(in_data):
    ret = {}
    for key, val in in_data.items():
        if not 'Tensor' in str(type(in_data[key][0])) :
            history_val = torch.tensor(val[0][['trea3','trea5', 'trea10']].to_numpy(np.single))
        else:
            history_val = val[0][:,:3] # only use three elements : ['trea3','trea5', 'trea10']
        addi1 = history_val[-1] - history_val[-5]
        addi2 = history_val[-1] - history_val[-7]
        addi3 = history_val[-1] - history_val[-10]
        addi4 = history_val[-1] - history_val[-3]
        addi5 = history_val[-3] - history_val[-7]
        addi6 = history_val[-3] - history_val[-10]
        addi7 = history_val[-5] - history_val[-7]
        addi8 = history_val[-5] - history_val[-10]
        addi = torch.cat((addi1, addi2, addi3, addi4, addi5, addi6, addi7, addi8), dim=-1)    
        ret[key] = addi
    return ret

In [685]:
batch_size = 64
""" # no additional
train_dset = Index_data(train_data, train_dates)
test_dset = Index_data(test_data, test_dates)
"""
train_dset = Index_data(train_data, train_dates, get_addi(train_data))
test_dset = Index_data(test_data, test_dates, get_addi(test_data))
train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=batch_size, shuffle=False)

In [686]:
print(len(test_dset))

1068


In [88]:
test_dset[0]

[tensor([[-0.3948, -0.4402, -0.4924],
         [-0.4127, -0.4342, -0.4758],
         [-0.4307, -0.4704, -0.5015],
         [-0.5139, -0.5590, -0.6024],
         [-0.3228, -0.2934, -0.2630],
         [-0.2105, -0.1324, -0.0520],
         [ 0.1041,  0.1996,  0.2508],
         [ 0.0592,  0.1091,  0.1407],
         [ 0.2951,  0.3022,  0.3058],
         [ 0.1491,  0.1392,  0.1315],
         [ 0.2884,  0.2720,  0.2893],
         [ 0.2727,  0.2700,  0.2783],
         [ 0.3064,  0.3062,  0.3150],
         [ 0.3243,  0.2901,  0.2783],
         [ 0.4861,  0.4410,  0.3976]]),
 tensor([1.6137, 1.7458, 1.9783]),
 tensor([0.4450, 0.4970, 0.5450]),
 '2016-12-15',
 tensor([1.8300, 1.8230, 1.7480, 1.7100, 1.7220, 1.7250, 1.7600, 1.7200, 1.7370,
         1.7310, 1.7090, 1.7350, 1.7160, 1.6620, 1.6450, 1.6950]),
 tensor([ 0.0000, -0.0383, -0.4481, -0.6557, -0.5902, -0.5738, -0.3825, -0.6011,
         -0.5082, -0.5410, -0.6612, -0.5191, -0.6230, -0.9180, -1.0109, -0.7377]),
 1.83,
 '2016-11-24',
 tensor([

In [697]:
from torch.autograd import Variable

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, T):
        # input size: number of underlying factors (81)
        # T: number of time steps (10)
        # hidden_size: dimension of the hidden state
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.T = T

        self.lstm_layer = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = 1)
        self.attn_linear = nn.Linear(in_features = 2 * hidden_size + T - 1, out_features = 1)

    def forward(self, input_data):
        # input_data: batch_size * T - 1 * input_size        
        input_weighted = Variable(input_data.data.new(input_data.size(0), self.T - 1, self.input_size).zero_())
        input_encoded = Variable(input_data.data.new(input_data.size(0), self.T - 1, self.hidden_size).zero_())
        # hidden, cell: initial states with dimention hidden_size
        hidden = self.init_hidden(input_data) # 1 * batch_size * hidden_size
        cell = self.init_hidden(input_data)
        # hidden.requires_grad = False
        # cell.requires_grad = False
        for t in range(self.T - 1):
            # Eqn. 8: concatenate the hidden states with each predictor
            x = torch.cat((hidden.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           cell.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           input_data.permute(0, 2, 1)), dim = 2) # batch_size * input_size * (2*hidden_size + T - 1)
            # Eqn. 9: Get attention weights
            x = self.attn_linear(x.view(-1, self.hidden_size * 2 + self.T - 1)) # (batch_size * input_size) * 1
            attn_weights = F.softmax(x.view(-1, self.input_size)) # batch_size * input_size, attn weights with values sum up to 1.
            # Eqn. 10: LSTM
            weighted_input = torch.mul(attn_weights, input_data[:, t, :]) # batch_size * input_size
            # Fix the warning about non-contiguous memory
            # see https://discuss.pytorch.org/t/dataparallel-issue-with-flatten-parameter/8282
            self.lstm_layer.flatten_parameters()
            output, lstm_states = self.lstm_layer(weighted_input.unsqueeze(0), (hidden, cell))
            hidden = lstm_states[0]
            cell = lstm_states[1]
            # Save output
            input_weighted[:, t, :] = weighted_input
            input_encoded[:, t, :] = hidden
        return input_weighted, input_encoded, output

    def init_hidden(self, x):
        # No matter whether CUDA is used, the returned variable will have the same type as x.
        return Variable(x.data.new(1, x.size(0), self.hidden_size).zero_()) # dimension 0 is the batch dimension

In [773]:
class Predictor_rnn(nets.Net):
    def __init__(self, downnet=None, dim_hiddens=[200, 200], 
                 loss=None, optimizer=None, device='cuda:2',
                 dim_input=100, bidirectional=True, time_step=15, lr=0.001):
        """
        net is consists of [embed, rnn, downnet]
        :param downnet: define downstream job
        """
        super(Predictor_rnn, self).__init__(loss=loss, device=device)
        
        encoder_dim_hidden, rnn_dim_hidden = dim_hiddens
        self.encoder = Encoder(dim_input, encoder_dim_hidden, time_step)
        self.encoder2 = Encoder(encoder_dim_hidden, encoder_dim_hidden, 
                                time_step)
        # this is optional, add another layer
        self.rnn = nn.LSTM(input_size=dim_input, hidden_size=rnn_dim_hidden,
                          num_layers=2, batch_first=True, bidirectional=bidirectional)
            
        if downnet is None:
            self.downnet = nets.get_MLP([rnn_dim_hidden, rnn_dim_hidden*2, 1], 
                                        dropout=0.35, end=True)
        else:
            self.downnet = downnet
        
        self.init_weights()
        
        self.nets = [self.encoder, self.encoder2, self.rnn, self.downnet]
        parms = None
        for net1 in self.nets:
            net1 = net1.to(device)
            if parms is None:
                parms = list(net1.parameters())
            else:
                parms += list(net1.parameters())
        self.optimizer = optimizer(parms, lr=lr)
            
    def set_train(self):
        for net1 in self.nets:
            net1.train()
        
    def set_eval(self):
        for net1 in self.nets:
            net1.eval()
    
    def init_weights(self):
        # check is there a way to initialize lstm parameters?
        for ii, layer in enumerate(self.downnet):
            if 'Linear' in str(layer):
                #torch.nn.init.xavier_uniform_(self.downnet[ii].weight)
                torch.nn.init.xavier_normal_(self.downnet[ii].weight)

    def forward(self, x, addi=('NULL')):
        input_weighted, input_encoded, output = self.encoder(x)
        # output, # [dim_seq=1, dim_batch, dim_hidden] 
        output = output.squeeze(0)
        """ test
        #out, hidden = self.rnn(x)
        out, hidden = self.rnn(input_weighted)
        out = out[:,-1] # choose last output
        """
        """
        out = output
        """
        #"""
        input_weighted, input_encoded, output = self.encoder2(input_encoded)
        output = output.squeeze(0)
        out = output
        #"""
        
        if 'Tensor' in str(type(addi)):
            out = torch.cat((out, addi),dim=1)
        out = self.downnet(out)
        return out.view(-1)

    def run_eval(self, data):
        self.set_eval()
        loss = 0
        outs = None
        tgts = None
        with torch.no_grad():
            for i_batch, data_batch in enumerate(data):
                data_in = data_batch[0][:,1:]
                mean_val = data_batch[1]
                range_val = data_batch[2]
                pred_date = data_batch[3]
                tgts_in = data_batch[5]
                tgt = tgts_in
                addi = data_batch[-1]
                if 'Tensor' in str(type(addi)):
                    addi = addi.to(self.device)
                
                data_in = data_in.to(self.device)
                tgt = tgt.to(self.device)
                
                out = self.forward(data_in, addi)
                    
                #print('loss', loss, self.loss)
                loss += self.loss(out, tgt).cpu().numpy()
                out = out.cpu().numpy()
                #print('loss===', loss)
                tgt = tgt.cpu().numpy()
                #print('tgt', tgt.shape)
                if outs is None:
                    outs = out
                    tgts = tgt
                else:
                    outs = np.concatenate((outs, out), axis=0)
                    tgts = np.concatenate((tgts, tgt), axis=0)
        loss /= 1.0*(i_batch+1)
        print('evaluate', 'loss', loss, 'accuracy : define function')
        return outs, tgts, loss
    
    def run_batch(self, i_batch, data_batch):
        self.optimizer.zero_grad()
        data_in = data_batch[0][:,1:]
        mean_val = data_batch[1]
        range_val = data_batch[2]
        pred_date = data_batch[3]
        tgts_in = data_batch[5]
        tgt = tgts_in
        addi = data_batch[-1]
        if 'Tensor' in str(type(addi)):
            addi = addi.to(self.device)
            
        data_in = data_in.to(self.device)
        tgt = tgt.to(self.device)
        
        out = self.forward(data_in, addi)
        
        loss = self.loss(out, tgt)
        loss.backward()
        self.optimizer.step()
        return loss.detach().cpu().item()

In [791]:
dim_input = train_dset[0][0].shape[-1]
time_step = train_dset[0][0].shape[0]
dim_hidden = 100
"""
# no addition
dim_hiddenb = dim_hidden
downnet = nets.get_MLP([dim_hiddenb, 2*dim_hidden, 1], dropout=0.0, end=True)
"""
#downnet = nets.get_MLP([dim_hidden+3*3, dim_hidden, 1], 
# Add difference variables
#"""
dim_hiddenb = dim_hidden+3*8
downnet = nets.get_MLP([dim_hiddenb, dim_hidden, 1], 
                        dropout=0.0, end=True)
#"""
#bidownnet = nets.get_MLP([dim_hidden*2+3*8, dim_hidden*2, 1], 
#                        dropout=0.0, end=True)
loss = nn.MSELoss() # which combines logsoftmax and nll loss
optimizer = optim.Adam
device = torch.device("cuda:2")

In [792]:
predictor = Predictor_rnn(loss=loss, optimizer=optimizer, 
                          device=device, dim_input=dim_input, 
                          dim_hiddens=[dim_hidden, dim_hidden], 
                          #downnet=None, bidirectional=False, 
                          downnet=downnet, bidirectional=False, lr=0.001,
                          time_step=env.input_size)
                          #dim_hidden=dim_hidden, downnet=bidownnet, bidirectional=True)

In [793]:
predictor.run_train(n_epoch=30, data=train_loader, test_data=test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 0 loss 0.09726834519585567
evaluate loss 0.1382761851829641 accuracy : define function
epoch 1 loss 0.09272779596608076
evaluate loss 0.13492741194718025 accuracy : define function
epoch 2 loss 0.0924056424245016
evaluate loss 0.13625193584491224 accuracy : define function
epoch 3 loss 0.09129118974973906
evaluate loss 0.13498407470829346 accuracy : define function
epoch 4 loss 0.09170094082382188
evaluate loss 0.13491127453744411 accuracy : define function
epoch 5 loss 0.09073380080621633
evaluate loss 0.1373136026017806 accuracy : define function
epoch 6 loss 0.09055503274300206
evaluate loss 0.13598577875424833 accuracy : define function
epoch 7 loss 0.09004736196861338
evaluate loss 0.13570796742158778 accuracy : define function
epoch 8 loss 0.08934091854451308
evaluate loss 0.13613879702547016 accuracy : define function
epoch 9 loss 0.08990806869384069
evaluate loss 0.1346574513351216 accuracy : define function
epoch 10 loss 0.09009381097310515
evaluate loss 0.13438879829995

In [785]:
rets = {}
tgts = None
preds = None
predictor.set_eval()
with torch.no_grad():
    for data_batch in test_loader:
        data_in = data_batch[0][:,1:]
        tgts_in = data_batch[5]
        tgt = tgts_in
        
        data_in = data_in.to(device)
        addi = data_batch[-1]
        if 'Tensor' in str(type(addi)):
            addi = addi.to(device)

        tgt = tgt.numpy()
        out = predictor.forward(data_in, addi).cpu().numpy()
        if preds is None:
            preds = out
            tgts = tgt
        else:
            preds = np.concatenate((preds, out), axis=0)
            tgts = np.concatenate((tgts, tgt), axis=0)
            
        prediction_dates = data_batch[3]
        past_prices = data_batch[6]
        predicting_dates = data_batch[7]
        for ii, date1 in enumerate(prediction_dates):
            out1 = out[ii]
            #out_orig = out1/10*past_prices[ii] + past_prices[ii]
            out_orig = out1*past_prices[ii]
            rets[date1] = {'predicting_date': predicting_dates[ii],
                          'pred': out_orig,
                           'past': past_prices[ii],
                           'tgt': data_batch[4][ii]}
            
preds = preds.reshape(-1)
tgts = tgts.reshape(-1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [786]:
limit = list(rets.keys()).index('2021-01-04')
sum1 = np.sum((preds[limit:] > 0) & (tgts[limit:] > 0))
sum2 = np.sum((preds[limit:] < 0) & (tgts[limit:] < 0))
print(sum1, np.sum(tgts[limit:]>0))
print(sum2, np.sum(tgts[limit:]<0))
print(len(preds[limit:]), sum1+sum2, 1.0*(sum1+sum2)/len(preds[limit:]))

36 44
7 24
71 43 0.6056338028169014


In [787]:
sum1 = np.sum((preds > 0) & (tgts > 0))
sum2 = np.sum((preds < 0) & (tgts < 0))
print(sum1, np.sum(tgts>0))
print(sum2, np.sum(tgts<0))
print(len(preds), sum1+sum2, 1.0*(sum1+sum2)/len(preds))

350 482
154 568
1068 504 0.47191011235955055


In [622]:
rets_2021 = {key: val for key, val in rets.items() if key>'2021-01-01'}
rows = list(rets_2021['2021-01-04'].keys())
cols = list(rets_2021.keys())

In [376]:
rets_2021b = {}
for col in cols:
    for row in rows:
        rets_2021b.setdefault(row, {})
        val = rets_2021[col][row]
        if type(val) is not str:
            rets_2021b[row][col] = rets_2021[col][row].item()        
        else:
            rets_2021b[row][col] = rets_2021[col][row]